In [8]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 49.1 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
 

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import transformers

# model = transformers.AutoModelForCausalLM.from_pretrained(
#   'mosaicml/mpt-7b',
#   trust_remote_code=True,
#   torch_dtype=torch.bfloat16,
    
# )
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
model_id = 'mosaicml/mpt-7b'
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, 
#                                              quantization_config=bnb_config, 
#                                              device_map={"":0},
#                                             trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    load_in_4bit=True, 
    device_map='cuda:0',
    trust_remote_code=True
)

model.eval()
# model.to("cuda:0")

/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/053e1a33a6e7043aefaa3f5d13c48269a5511cff/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/container/ucx/lib64'), PosixPath('/usr/lib/libibverbs'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib/libibverbs:/container/ucx/lib:/container/ucx/lib64:/container/ompi/lib:/


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.76s/it]


MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear4bit(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear4bit(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [3]:
transformers.__version__

'4.30.0.dev0'

In [8]:
!nvidia-smi

Thu Jun  1 21:22:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:18:00.0 Off |                    0 |
| N/A   39C    P0    32W /  70W |   7308MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:19:00.0 Off |                    0 |
| N/A   

In [4]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Modelsize: {model_size/1000**2:.1f}M parameters")

Modelsize: 3428.1M parameters


In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained('mosaicml/mpt-7b')

In [6]:
txt = """\
image_path = "/mnt/image.png"

# load image
"""
tokenized_example = tokenizer(txt, return_tensors='pt')

In [7]:
tokenized_example['input_ids']

tensor([[ 5695,    64,  3967,   426, 13357,    78,  2649,    16,  5695,    15,
          8567,     3,   187,   187,     4,  3301,  2460,   187]])

In [8]:
outputs = model.generate(tokenized_example['input_ids'], max_new_tokens=150, do_sample=False, top_k=5, top_p=0.95)


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [9]:
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(answer[0].rstrip())

image_path = "/mnt/image.png"

# load image
image = Image.open(image_path)

# get image size
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image.size

# get image size in pixels
width, height = image


In [10]:
answer_text = """\
from PIL import Image

image = Image.read(image_path)
"""
tokenized_answer = tokenizer.encode(answer_text ,return_tensors='pt')

In [11]:
tokenized_answer.shape

torch.Size([1, 18])

In [17]:
outputs = model(**tokenized_example.to("cuda:0"))

In [18]:
outputs.logits.shape

torch.Size([1, 18, 50432])

In [19]:
tokenized_example['input_ids'].shape

torch.Size([1, 18])

In [23]:
outputs.logits

tensor([[[ 14.8984,  -0.6685,  16.4375,  ...,  -0.7480,  -0.7554,  -0.7432],
         [ 25.2188,   9.3594,  25.2812,  ...,   9.3281,   9.3281,   9.3359],
         [  8.3281, -10.2891,  10.3047,  ..., -10.5312, -10.5469, -10.5312],
         ...,
         [ 29.0625,  10.9609,  31.4062,  ...,  10.8047,  10.7969,  10.8047],
         [ 39.2500,  18.0000,  41.5312,  ...,  17.8125,  17.8125,  17.8125],
         [ 44.2812,  18.5312,  43.6875,  ...,  18.1875,  18.1875,  18.1875]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>)

In [24]:
last_token_output = outputs.logits.view(1,-1)
last_token_output.shape

torch.Size([1, 907776])

In [25]:
torch.argmax(last_token_output)

tensor(554939, device='cuda:0')

In [26]:
tokenized_answer.shape
labels = tokenized_answer[0][0].view(1)
labels

tensor([4064])

In [27]:
for name, param in model.named_parameters():
    print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
    if "31" not in name:
        param.requires_grad = False
    print(name, param.requires_grad)

transformer.wte.weight   Modelsize: 206.6M parameters
transformer.wte.weight False
transformer.blocks.0.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_1.weight False
transformer.blocks.0.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.0.attn.Wqkv.weight False
transformer.blocks.0.attn.out_proj.weight   Modelsize: 16.8M parameters
transformer.blocks.0.attn.out_proj.weight False
transformer.blocks.0.norm_2.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_2.weight False
transformer.blocks.0.ffn.up_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.up_proj.weight False
transformer.blocks.0.ffn.down_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.down_proj.weight False
transformer.blocks.1.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.1.norm_1.weight False
transformer.blocks.1.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.1.attn.Wqkv.weight False
transformer.blocks.1.a

In [28]:
params = sum(t.numel() for t in model.transformer.blocks[-1].parameters())
print(f"Modelsize: {params/1000**2:.1f}M parameters")

Modelsize: 201.3M parameters


In [29]:
lossfct = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
last_token_output[0].shape


torch.Size([907776])

In [31]:
labels = tokenized_answer[0][0].view(1)
print(labels.dtype, labels.shape,last_token_output.float().dtype)
loss = lossfct(last_token_output.float().to("cuda:0"),labels.to("cuda:0"))

torch.int64 torch.Size([1]) torch.float32


In [32]:
loss.item()

41.046722412109375

In [33]:
model.train()
loss.backward()
optimizer.step()
optimizer.zero_grad(set_to_none=True)


In [ ]:
outputs = model.generate(tokenized_example['input_ids'].to('cuda:0'), max_new_tokens=50, do_sample=False, top_k=5, top_p=0.95)


In [ ]:
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(answer[0].rstrip())

image_path = "/mnt/image.png"

# load image
img = cv2.imread(image_path)

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# blur
blur =
